In [ ]:
mode ="gpu"  # gpu or cpu

In [10]:
from numba import jit, cuda
import numpy as np
from numba import float32


@jit
def calculate_cost(solution: np.ndarray) -> float:
    # Implement your cost function here
    # Example: return the squared magnitude of the solution
    return np.sum(solution ** 2)

@jit
def generate_neighbor_cpu(current_solution:np.ndarray):
    # Implement neighbor generation based on your problem
    # Example: perturb the current solution by adding random noise
    perturbation = np.random.randn(*current_solution.shape)
    return current_solution + perturbation

@cuda.jit
def generate_neighbor(current_solution:np.ndarray, random):
    # Implement neighbor generation based on your problem
    # Example: perturb the current solution by adding random noise
    pos = cuda.grid(1)
    if pos < current_solution.size:
        current_solution[pos] += random

@jit
def update_temperature(current_temperature):
    # Implement your cooling schedule here
    # Example: exponential decay
    cooling_rate = 0.95
    return current_temperature * cooling_rate



C:\Users\Kleindienst\AppData\Local\Temp\ipykernel_20304\14628636.py:7: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def calculate_cost(solution: np.ndarray) -> float:
C:\Users\Kleindienst\AppData\Local\Temp\ipykernel_20304\14628636.py:21: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def update_temperature(current_temperature):


In [15]:
initial_solution = np.random.random(200)

In [16]:
%%timeit
current_solution = initial_solution
current_cost = calculate_cost(current_solution)
best_solution = current_solution
best_cost = current_cost
current_temperature = 10

threadsperblock = 32
blockspergrid = (current_solution.size + (threadsperblock - 1)) // threadsperblock

for iteration in range(1000):
    
    if mode == "gpu":
        new_solution = current_solution.copy()
        generate_neighbor[blockspergrid,threadsperblock](new_solution, np.random.randn())
    else:
        new_solution = generate_neighbor_cpu(current_solution)
    new_cost = calculate_cost(new_solution)
    delta_cost = new_cost - current_cost

    if delta_cost < 0 or np.random.random(1) < np.exp(-delta_cost / current_temperature):
        current_solution = new_solution
        current_cost = new_cost

        if new_cost < best_cost:
            best_solution = new_solution
            best_cost = new_cost

    # Update temperature according to the annealing schedule
    current_temperature = update_temperature(current_temperature)

c:\Users\Kleindienst\Anaconda3\envs\prognos\lib\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


564 ms ± 46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Numpy only
12.2 ms ± 315 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
### Jit
10.5 ms ± 329 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
### Jit on Cuda
598 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
best_solution

NameError: name 'best_solution' is not defined